<hr>

# 🤖 MACHINE LEARNING 🤖

<style>
h1 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

In [28]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, average_precision_score

In [5]:
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test  = pd.read_csv("../data/processed/X_test.csv")

y_train = pd.read_csv("../data/processed/y_train.csv").values.ravel()
y_test  = pd.read_csv("../data/processed/y_test.csv").values.ravel()

# 1. KNearest Neighbours model

## Fitting and prediction

In [19]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train) #fit the model to the training data
pred = knn.predict(X_test) #make predictions on the test set using the fitted model

## Results 

### Accuracy

In [20]:
knn.score(X_test, y_test)

0.8450932684509327

### Classification Report

In [ ]:
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))


[[1960   95]
 [ 287  124]]
              precision    recall  f1-score   support

       False       0.87      0.95      0.91      2055
        True       0.57      0.30      0.39       411

    accuracy                           0.85      2466
   macro avg       0.72      0.63      0.65      2466
weighted avg       0.82      0.85      0.82      2466



- 287 actual buyers were predicted as non-buyers
- Only 124 buyers were correctly predicted
- Hence the precision and recall rate for minority class, TRUE, is very low.

### ROC AUC

In [29]:
# Get probability of positive / minority class (class = 1)
probs = knn.predict_proba(X_test)[:, 1]

# ROC-AUC
roc_auc = roc_auc_score(y_test, probs)

print("ROC-AUC:", roc_auc)

ROC-AUC: 0.7078208156475512


### PR AUC

In [30]:
# PR-AUC
pr_auc = average_precision_score(y_test, probs)
print("PR-AUC :", pr_auc)

PR-AUC : 0.3661417550958389


## Potential improvements:
- Try class weighted methods
- SMOTE
- Logistic regression with class weighted - **TRIED as 1st option**
- Probability models (with Threshold)

## 2. Logistic Regression (class weight balanced)

## Fitting and prediction

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Initialize model
lr= LogisticRegression(class_weight='balanced',max_iter=5000)

# Train
lr.fit(X_train, y_train)

# Predict
y_pred = lr.predict(X_test)


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Results 

### Classification Report

In [23]:
# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1852  203]
 [ 108  303]]
              precision    recall  f1-score   support

       False       0.94      0.90      0.92      2055
        True       0.60      0.74      0.66       411

    accuracy                           0.87      2466
   macro avg       0.77      0.82      0.79      2466
weighted avg       0.89      0.87      0.88      2466



# Comparison b/w KNN and Log.Regression

In [24]:
import pandas as pd
from sklearn.metrics import classification_report

# Get reports as dictionary
knn_report = classification_report(y_test, pred, output_dict=True)
log_report = classification_report(y_test, y_pred, output_dict=True)

# Create comparison dataframe
comparison = pd.DataFrame({
    "Metric": [
        "Accuracy",
        "Precision_TRUE",
        "Recall_TRUE",
        "F1_TRUE",
        "Macro_F1",
        "Weighted_F1"
    ],
    "KNN": [
        knn_report["accuracy"],
        knn_report["True"]["precision"],
        knn_report["True"]["recall"],
        knn_report["True"]["f1-score"],
        knn_report["macro avg"]["f1-score"],
        knn_report["weighted avg"]["f1-score"],
    ],
    "Logistic Regression": [
        log_report["accuracy"],
        log_report["True"]["precision"],
        log_report["True"]["recall"],
        log_report["True"]["f1-score"],
        log_report["macro avg"]["f1-score"],
        log_report["weighted avg"]["f1-score"],
    ]
})

comparison


,Metric,KNN,Logistic Regression
0,Accuracy,0.845093,0.873885
1,Precision_TRUE,0.566210,0.598814
2,Recall_TRUE,0.301703,0.737226
3,F1_TRUE,0.393651,0.660851
4,Macro_F1,0.652427,0.791696
5,Weighted_F1,0.824945,0.878925
